# Import the Required Libraries

In [5]:
import random
import simpy
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from faker import Faker

In [6]:
# Initialize Faker for realistic names
fake = Faker()

# Configuration

In [7]:
CONFIG = {
    'simulation': {
        'beds_capacity': 10,
        'max_queue': 5,
        'sim_duration': 30 * 24,  # 720 hours (30 days)
        'arrival_interval': (1, 3),
    }, 
    # various diseases, age groups & their specifications
    'diseases': { 
        'Broken Arm': {'duration': (12, 24), 'base_severity': 2, 'subtypes': {
            'Hairline Fracture': 1, 'Simple Fracture': 2, 'Compound Fracture': 4}},
        'Concussion': {'duration': (18, 24), 'base_severity': 4, 'subtypes': {
            'Mild': 3, 'Moderate': 4, 'Severe': 5}},
        'Simple Fracture': {'duration': (12, 18), 'base_severity': 3, 'subtypes': {
            'Non-displaced': 2, 'Displaced': 3, 'Comminuted': 4}},
        'Appendicitis': {'duration': (12, 24), 'base_severity': 5, 'subtypes': {
            'Early Stage': 4, 'Acute': 5, 'Perforated': 6}},
        'Pneumonia': {'duration': (18, 24), 'base_severity': 4, 'subtypes': {
            'Mild': 3, 'Moderate': 4, 'Severe': 5}}
    },
    'age_groups': {
        '0-18': {'probability': 0.15, 'priority': 3, 'factors': ['pediatric']},
        '19-35': {'probability': 0.25, 'priority': 2, 'factors': ['young_adult']},
        '36-55': {'probability': 0.35, 'priority': 1, 'factors': ['adult']},
        '56-75': {'probability': 0.15, 'priority': 4, 'factors': ['elderly', 'comorbidities']},
        '76+': {'probability': 0.10, 'priority': 5, 'factors': ['geriatric', 'high_risk']}
    }
}

In [9]:
# utility function for converting simulation hours to a timestamp string
def hours_to_timestamp(hours):
    date = datetime(2025, 1, 1)
    delta = timedelta(hours=hours)
    return (date + delta).strftime('%Y-%m-%d %H:%M:%S')


# Simulation Processes

In [ ]:
def create_patient(env, disease, age_group):
    """Create a patient with attributes based on disease and age group."""
    subtype = random.choice(list(CONFIG['diseases'][disease]['subtypes'].keys()))
    mins, maxs = CONFIG['diseases'][disease]['duration']
    base_severity = CONFIG['diseases'][disease]['base_severity']
    subtype_severity = CONFIG['diseases'][disease]['subtypes'][subtype]
    
    vital_signs = {
        'bp': random.randint(90, 180),
        'pulse': random.randint(50, 120),
        'temp': round(random.uniform(36.5, 39.5), 1)
    }
    
    score = base_severity + (subtype_severity - 3)
    medical_factors = CONFIG['age_groups'][age_group]['factors'].copy()
    
    if vital_signs['bp'] > 160 or vital_signs['bp'] < 100:
        score += 0.5
        medical_factors.append('abnormal_bp')
    if vital_signs['temp'] > 38:
        score += 0.5
        medical_factors.append('fever')
    
    priority = (score * 0.6) + (CONFIG['age_groups'][age_group]['priority'] * 0.4)
    adjustment = 1 + (6 - score) / 20
    treatment_duration = round(random.uniform(mins, maxs) * adjustment)
    
    return {
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'age_group': age_group,
        'disease': disease,
        'disease_subtype': subtype,
        'vital_signs': vital_signs,
        'status': 'pending',
        'rejection_reason': None,
        'bed_number': None,
        'arrival_time': env.now,
        'arrival_timestamp': hours_to_timestamp(env.now),
        'treatment_duration': treatment_duration,
        'discharge_time': None,
        'discharge_timestamp': None,
        'severity_score': score,
        'priority_level': priority,
        'medical_factors': medical_factors,
        'evaluation_time': 0,
        'admission_delay': 0
    }

def patient_flow(env, beds, patient_records):
    """Manage the patient admission process."""
    age_group = random.choices(
        list(CONFIG['age_groups'].keys()),
        weights=[g['probability'] for g in CONFIG['age_groups'].values()]
    )[0]
    disease = random.choice(list(CONFIG['diseases'].keys()))
    patient = create_patient(env, disease, age_group)
    
    base_wait = max(0.1, 4 - (patient['priority_level'] / 2))
    wait = random.uniform(base_wait / 2, base_wait * 1.5)
    
    with beds.request() as request:
        yield env.timeout(random.uniform(0.1, 0.5))
        result = yield request | env.timeout(wait)
        
        patient['evaluation_time_min'] = int(round(wait * 60))
        
        if request not in result:
            rejection_reasons = {
                'low_priority': 0.6 - (patient['priority_level'] * 0.1),
                'non_urgent': 0.3 if patient['severity_score'] < 3 else 0.1,
                'resource_constraints': 0.4 if beds.count / beds.capacity > 0.9 else 0.1,
                'transfer_recommended': 0.2 if patient['severity_score'] > 5 else 0
            }
            probable_reasons = {k: v for k, v in rejection_reasons.items() if random.random() < v}
            rejection_reason = max(probable_reasons.items(), key=lambda x: x[1])[0] if probable_reasons else 'capacity'
            
            patient.update({
                'status': 'rejected',
                'rejection_reason': rejection_reason
            })
            patient_records.append(patient)
            return
        
        delay = random.uniform(0.2, 1.0)
        yield env.timeout(delay)
        
        available_beds = [num for num, status in beds.bed_status.items() if status == 'available']
        
        bed_number = random.choice(available_beds)
        beds.bed_status[bed_number] = {
            'patient_id': f"{patient['first_name']}_{patient['last_name']}",
            'admission_time': env.now,
            'discharge_time': env.now + patient['treatment_duration']
        }
        
        patient.update({
            'status': 'admitted',
            'bed_number': bed_number,
            'discharge_time': env.now + patient['treatment_duration'],
            'discharge_timestamp': hours_to_timestamp(env.now + patient['treatment_duration']),
            'admission_delay_min': int(round(delay * 60))
        })
        patient_records.append(patient)
        
        yield env.timeout(patient['treatment_duration'])
        beds.bed_status[bed_number] = 'available'

def patient_generator(env, beds, patient_records):
    """Generate patients at random intervals."""
    while True:
        yield env.timeout(random.uniform(*CONFIG['simulation']['arrival_interval']))
        env.process(patient_flow(env, beds, patient_records))
